# Multi-Objective Resource Allocation Method for IoT applications in the context of Mobile Edge Computing (MEC)

## Dépendances
Import des dépendances Python nécessaires au projet.

In [1]:
import numpy as np

## Dataset
Cette partie décrit le dataset et les données utilisées pour répondre au problème décrit plus bas.

## <p style="color:red;">Partie à remplacer par le vrai dataset de Kaggle</p>

In [ ]:
# Définition des variables
n_taches = 100  # Nombre de tâches
n_noeuds = 25  # Nombre de serveurs

phi = np.random.rand(n_taches,n_noeuds)

# Capacités de traitement maximales
C = np.array([10, 12, 8, 15])

# Puissance consommée par tâche
P = np.random.rand(n_taches, n_noeuds)

# Temps de processing par tâche
t = np.random.rand(n_taches, n_noeuds)

# Charge de travail par tâche
w = np.random.rand(n_taches)

# Paramètres de l'algorithme Firefly
alpha = 0.5  # Coefficient d'absorption
beta = 0.2  # Coefficient de randomisation
gamma = 1.0  # Facteur d'atténuation

# Variable pour fonction objectives
rx = 0.2
rf = 0.125
rn = 0.5
rg = 5
ZYS = 3
sigma = 5
wk = {0:0.75, 1:1.25, 2:0.3, 3:1.5}
p = 1
q = 2
r = 3

g = 5
M = 5

### Paramètres expérimentaux

In [ ]:
# Paramètre de nombre d'applications IoT
nb_app = [50, 100, 150, 200, 250]
# Paramètre du nombre de VM par serveur MEC
nb_vm = 8
# Paramètre de la quantité de VM nécessaire par application
nb_vm_app = [1,7]

### RAS de départ

In [ ]:
# Wattage des serveurs
pr_serv = 350
# Wattage des VM occupées
pr_ovm = 75
# Wattage des VM inoccupées
pr_uvm = 45

## Définition du problème dans le code

### Définition des contraintes

In [ ]:
def constraint1(x):
    # Le nombre de vm obtenue par iot ne doit pas exceder le nombre de machines inactive sur le serveur
    return np.array([np.sum(x[i::n_noeuds]) for i in range(n_taches)]) - 1

def constraint2(x):
    # La capacité de traitement maximale de chaque nœud ne doit pas être dépassée
    return C - np.dot(w, x.reshape((n_taches, n_noeuds)))

### Définition des Fonctions Objectifs

In [ ]:
#     return np.sum(position * demandes_taches * capacites_serveurs)
def f1_energy(position):
  """
  Calcule la consommation d'énergie totale des serveurs MEC.

  Args:
    variables: Un tableau contenant les valeurs des variables de décision.

  Returns:
    La consommation d'énergie totale (EC).
  """
  # Décodage des variables
  wnt = position[:72]  # Puissance allouée aux tâches
  atn = position[72:144]  # Type d'antenne utilisé pour chaque tâche
  rn = position[144:]  # Débit de données pour chaque tâche
  # Calcul de la consommation d'énergie de base (ECB)
  ecb = 0
  for i in range(72):
    ecb += wnt[i] * rx
  # Calcul de la consommation d'énergie liée au front-haul (ECF)
  ecf = 0
  for n in range(12):
    for m in range(72):
      ecf += wnt[m] * rf * phi[atn[m], n]
  # Calcul de la consommation d'énergie liée au backhaul (ECK)
  eck = 0
  for n in range(12):
    for m in range(72):
      eck += (ZYS - sum(phi[atn[m], n] for atn in atn)) * wnt[m] * rn
  # Consommation d'énergie totale
  ec = ecb + ecf + eck
  return ec

#     return np.max(np.sum(position * demandes_taches, axis=1))

def f2_time(position):
  """
  Calcule le temps de complétion total des tâches.

  Args:
    variables: Un tableau contenant les valeurs des variables de décision.

  Returns:
    Le temps de complétion total (TE).
  """

  # Décodage des variables
  x = position  # Affectation des variables
  # Calcul du temps de traitement (TM)
  tm = 0
  for j in range(3):
    for m in range(72):
      if j == 0:
        tm += x[m] * g / wk[p]
      elif j == 1:
        tm += (x[m] * g / wk[p]) + (x[m] * g / wk[q])
      else:
        tm += (x[m] * g / wk[p]) + (x[m] * g / wk[q]) + (x[m] * g / wk[r])
  # Calcul du temps d'attente (TW)
  tw = 0
  for n in range(12):
    for m in range(72):
      tw += x[m] * sum(1 / wk[p] for p in range(12) if p != n)
  # Calcul du temps de transfert (TB)
  tb = 0
  for j in range(3):
    for m in range(72):
      if j == 0:
        tb += x[m] * rg / wk[n]
      elif j == 1:
        tb += (x[m] * rg / wk[p]) + (x[m] * rg / wk[g])
      else:
        tb += (x[m] * rg / wk[p]) + (x[m] * rg / wk[q]) + (x[m] * rg / wk[r])
  # Temps de complétion total
  te = tm + tw + tb
  return te


In [ ]:
#Contraintes

def c1_nbVM(position):
    """
    Vérifie si le nombre de VMs actives par serveur MEC est inférieur ou égal à M.
    Args:
        variables: Un tableau contenant les valeurs des variables de décision.

    Returns:
        True si la contrainte est satisfaite, False sinon.
    """
    # Décodage des variables
    x = position
    # Calcul du nombre de VMs actives par serveur MEC
    nb_vms_par_serveur = np.sum(x.reshape((12, 72)), axis=1)
    # Vérification de la contrainte
    return np.all(nb_vms_par_serveur <= M)

def c2_ressources(position):
    """
    Vérifie si la charge de chaque serveur MEC est inférieure ou égale à sa capacité maximale.

    Args:
        variables: Un tableau contenant les valeurs des variables de décision.

    Returns:
        True si la contrainte est satisfaite, False sinon.
    """
    # Décodage des variables
    x = position
    # Calcul de la charge de chaque serveur MEC
    charge_serveur = np.dot(x, sigma)
    # Vérification de la contrainte
    return np.all(charge_serveur <= capacites_serveurs[men])


In [ ]:
def fitness(position, w1, w2):
  # Consommation d'énergie
  energie = f1_energy(position)

  # Temps d'exécution
  temps = f2_time(position)

  # Fonction multiobjectives pondérée (c'est comme ça qu'on rend firefly multi-objectif)
  return w1 * energie + w2 * temps


constraints = [{'type': 'eq', 'fun': c1_nbVM},
               {'type': 'ineq', 'fun': c2_ressources}]

In [ ]:
def validation(position, n_serveurs, demandes_taches, capacites_serveurs, nb_vm_inactives):
  # Contrainte de ressources
  for i in range(n_serveurs):
    if np.sum(position[:, i] * demandes_taches) > capacites_serveurs[i]:
      return False

  # Contrainte de VM
  nb_vm_par_serveur = np.sum(position, axis=0)
  for i in range(n_serveurs):
    if nb_vm_par_serveur[i] > nb_vm_inactives[i]:
      return False

  return True


In [ ]:
def firefly_algorithm(
  n_taches,
  n_serveurs,
  capacites_serveurs,
  demandes_taches,
  nb_vm_inactives,
  w1=1,
  w2=1,
  w3=1,
  alpha,
  beta,
  gamma,
  iterations
  ):

  """
  Fonction implémentant l'algorithme Firefly pour l'optimisation de l'allocation de VM.

  Args:
    n_taches: Nombre de tâches.
    n_serveurs: Nombre de serveurs.
    capacites_serveurs: Capacités des serveurs.
    demandes_taches: Demandes des tâches.
    nb_vm_inactives: Nombre de VM inactives par serveur.
    w1: Poids de la consommation d'énergie.
    w2: Poids du temps d'exécution.
    w3: Poids du déséquilibre de charge.
    alpha: Coefficient d'absorption.
    beta: Coefficient de randomisation.
    gamma: Facteur d'atténuation.
    iterations: Nombre d'itérations.

  Returns:
    Meilleure position des lucioles et la meilleure valeur de la fonction d'objectif.
  """

  # Initialisation des positions des lucioles
  positions = np.random.randint(0, 2, size=(n_taches, n_serveurs))

  # Initialisation des intensités des lucioles
  intensities = np.zeros(n_taches)

  for iteration in range(iterations):

    # Mise à jour des intensités des lucioles
    intensities = np.zeros(n_taches)
    for i in range(n_taches):
      if validation(positions[i], n_serveurs, demandes_taches, capacites_serveurs, nb_vm_inactives):
        intensities[i] = fitness(positions[i])

    # Mise à jour des positions des lucioles
    for i in range(n_taches):
      if validation(positions[i]):
        for j in range(n_serveurs):
          for k in range(n_taches):
            if k != i and validation(positions[i]):
              # Calcul de la distance entre les lucioles i et k
              distance = np.linalg.norm(positions[i] - positions[k])

              # Attraction vers la luciole la plus brillante
              attraction = alpha * intensities[k] * np.exp(-gamma * distance**2)

              # Perturbation aléatoire
              perturbation = beta * np.random.rand()

              # Mise à jour de la position de la luciole
              positions[i][j] += attraction + perturbation

  # Evaluation de la meilleure solution
  best_position = positions[np.argmin(intensities)]
  best_fitness = fitness(best_position)

  return best_position, best_fitness

